In [1]:
#from cabrillo.parser import parse_log_text
#import dateutil
#from datetime import date
#import re
#import xlrd
#import csv
#from collections import Counter
import maidenhead as mh
import folium
import json

In [2]:
with open('FMqsos.txt') as json_file:
    qsos = json.load(json_file)
            

In [3]:

def add_qso_data(m, qsos=qsos, plotdx=False, plotverify=False, edgecolor='green', nodemarkers=False):
    nodes = []
    for qso in qsos:        
        #print(qso)
        if len(qso['locator']) == 6: 
            ql1 = qso['locator'].upper()
        elif len(qso['qrz_locator']) == 6:
            ql1 = qso['qrz_locator'].upper()
        else:
            continue
        if len(qso['dx_locator']) == 6: 
            ql2 = qso['dx_locator'].upper()
        elif len(qso['qrz_dx_locator']) == 6:
            ql2 = qso['qrz_dx_locator'].upper()
        else:
            continue
            
        if ql1 == "AA00AA" or ql2 == "AA00AA":
            # qrz default
            continue
        #if qso['call'] == "EI9JA":print(qso)
        nodes.append(ql1)
        nodes.append(ql2)
        try:
            l1 = mh.toLoc(ql1)
            l2 = mh.toLoc(ql2)
        except ValueError:
            pass
            #print(qso)
            #print(qso['qrz_locator'], qso['qrz_dx_locator'])
        if qso['verified'] == True and plotverify:
            folium.PolyLine([l1,l2], color="red", weight=1, opacity=1).add_to(m)
        else:
            folium.PolyLine([l1,l2], color=edgecolor, weight=1, opacity=1).add_to(m)



    entrants=set([(x['call'],x['qrz_locator']) for x in qsos])
    locs = set(x[1].upper() for x in entrants)
    e2=[]
    for l in locs:
        e2.append((" ".join([e[0] for e in entrants if l in e[1].upper()]), l))
    e2

    if nodemarkers:
        for e in e2:
            folium.Marker(
                mh.toLoc(e[1]),
                popup=e[0],
                icon=folium.Icon(icon='wifi', prefix="fa", color='red')
            ).add_to(m)

    if plotdx:
        dxs=set([(x['dx_call'], x['qrz_dx_locator']) for x in qsos if 
                 x['qrz_dx_locator'] and 
                 x['dx_call'] not in [y[0] for y in entrants] and
                 x['qrz_dx_locator'] not in [y[1] for y in entrants]]
               )
        print(set(nodes) - set([x[1] for x in entrants]) - set([x[1]for x in dxs]))

        for e in dxs:
            folium.Marker(
                mh.toLoc(e[1]),
                popup=e[0],
                icon=folium.Icon(icon='user', prefix="fa")
            ).add_to(m)
    return m


In [4]:
import json
with open('FMqsos.txt') as json_file:
    fmqsos = json.load(json_file)
with open('FT8qsos.txt') as json_file:
    ft8qsos = json.load(json_file)
m = folium.Map(location=[53.4, -7.8],
               zoom_start=7,
               tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}.png',
               attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

add_qso_data(m, fmqsos, nodemarkers=True)
#add_qso_data(m, ft8qsos, edgecolor='blue')
m.save('indexFM.html')
m

In [5]:
with open('FT8qsos.txt') as json_file:
    qsos = json.load(json_file)
m = folium.Map(location=[52.7, -5.5],
               zoom_start=6,
               tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}.png',
               attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

add_qso_data(m, qsos, nodemarkers=True)
m.save('indexFT8.html')
m